In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Fri Oct 14 04:22:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    44W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
%cd /content/drive/MyDrive/colab/distillog/Spirit-kd-chronological

/content/drive/MyDrive/colab/distillog/Spirit-kd-chronological


In [4]:
!pip install torchinfo
!pip install overrides

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import Parameter
from torch.nn.modules.module import Module
from tqdm import tqdm
import math
import csv
from time import time 
from torchinfo import summary
from utils import save_model, train, read_data, load_data
from utils import DistilLog
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_classes = 2
batch_size = 50
learning_rate = 0.0005
hidden_size = 128
input_size = 30
sequence_length = 50
num_layers = 2

train_path = '../datasets/Spirit/chronological_train.csv'
save_teacher_path = '../datasets/Spirit/model/chronological_teacher.pth'
save_noKD_path = '../datasets/Spirit/model/chronological_noKD.pth'

Teacher = DistilLog(input_size, hidden_size, num_layers , num_classes, is_bidirectional=False).to(device)
noKD = DistilLog(input_size = input_size, hidden_size = 4, num_layers = 1, num_classes = num_classes, is_bidirectional=False).to(device)
#summary(Teacher, input_size=(50, 50, 30))

train_x, train_y = read_data(train_path, input_size, sequence_length)
train_loader = load_data(train_x, train_y, batch_size)

#Teacher = train(Teacher, train_loader, learning_rate = 0.0003, num_epochs = 400)
#save_model(Teacher, save_teacher_path)



/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [ ]:
Teacher = train(Teacher, train_loader, learning_rate = 0.0003, num_epochs = 200)
save_model(Teacher, save_teacher_path)

Train Epoch: 1/200 [78400/79999 (100%)]  Loss: 425.424866: 100%|██████████| 1600/1600 [00:07<00:00, 207.28it/s]
Train Epoch: 2/200 [78400/79999 (100%)]  Loss: 246.409735: 100%|██████████| 1600/1600 [00:07<00:00, 225.32it/s]
Train Epoch: 3/200 [78400/79999 (100%)]  Loss: 148.472836: 100%|██████████| 1600/1600 [00:06<00:00, 231.12it/s]
Train Epoch: 4/200 [78400/79999 (100%)]  Loss: 119.987252: 100%|██████████| 1600/1600 [00:07<00:00, 228.55it/s]
Train Epoch: 5/200 [78400/79999 (100%)]  Loss: 108.362268: 100%|██████████| 1600/1600 [00:06<00:00, 231.25it/s]
Train Epoch: 6/200 [78400/79999 (100%)]  Loss: 96.343384: 100%|██████████| 1600/1600 [00:06<00:00, 230.57it/s]
Train Epoch: 7/200 [78400/79999 (100%)]  Loss: 93.572859: 100%|██████████| 1600/1600 [00:07<00:00, 227.37it/s]
Train Epoch: 8/200 [78400/79999 (100%)]  Loss: 87.779109: 100%|██████████| 1600/1600 [00:06<00:00, 231.09it/s]
Train Epoch: 9/200 [78400/79999 (100%)]  Loss: 85.029936: 100%|██████████| 1600/1600 [00:06<00:00, 229.21it

In [6]:
from utils import load_data, load_model
test_path = '../datasets/Spirit/chronological_test.csv'
split = 50

fi = pd.read_csv('../datasets/Spirit/pca_vector.csv', header = None)
vec = []
vec = fi
vec = np.array(vec)

test_logs_series = pd.read_csv(test_path)
test_logs_series = test_logs_series.values
test_total = len(test_logs_series)
sub = int(test_total/split)



def mod(l, n):
    """ Truncate or pad a list """
    r = l[-1*n:]
    if len(r) < n:
        r.extend(list([0]) * (n - len(r)))
    return r

def load_test(i):
    if i!=split-1:
        label = test_logs_series[i*sub:(i+1)*sub,1]
        logs_data = test_logs_series[i*sub:(i+1)*sub,0]
    else:
        label = test_logs_series[i*sub:,1]
        logs_data = test_logs_series[i*sub:,0]
    logs = []

    for logid in range(0,len(logs_data)):
        ori_seq = [
            int(eventid) for eventid in logs_data[logid].split()]
        seq_pattern = mod(ori_seq, sequence_length)
        vec_pattern = []

        for event in seq_pattern:
            if event == 0:
                vec_pattern.append([-1]*input_size)
            else:
                vec_pattern.append(vec[event-1])  
        logs.append(vec_pattern)
    logs = np.array(logs)
    train_x = logs
    train_y = label
    train_x = np.reshape(train_x, (train_x.shape[0], -1, input_size))
    train_y = train_y.astype(int)
    return train_x, train_y



def test(model, criterion = nn.CrossEntropyLoss()):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        TP = 0 
        FP = 0
        FN = 0 
        TN = 0
        for i in range (0, split):        #################################################
            test_x, test_y = load_test(i)
            test_loader = load_data(test_x, test_y, batch_size)            
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output, _ = model(data)
                test_loss += criterion(output, target) # sum up batch loss
                
                output = torch.sigmoid(output)[:, 0].cpu().detach().numpy()
                predicted = (output < 0.2).astype(int)
                target = np.array([y.cpu() for y in target])

                TP += ((predicted == 1) * (target == 1)).sum()
                FP += ((predicted == 1) * (target == 0)).sum()
                FN += ((predicted == 0) * (target == 1)).sum()
                TN += ((predicted == 0) * (target == 0)).sum()
        P = 100 * TP / (TP + FP)
        R = 100 * TP / (TP + FN)
        F1 = 2 * P * R / (P + R)   
        accuracy = 100 * (TP + TN)/(TP + TN + FP + FN)
        #MCC = 100*(TP*TN + FP*FN)/math.sqrt((TP+FP)*(TN+FN)*(TN+FP)*(TP+FN))         
    return accuracy, test_loss, P, R, F1, TP, FP, TN, FN

Teacher = load_model(Teacher, save_teacher_path)


start_time = time()
accuracy, test_loss, P, R, F1, TP, FP, TN, FN = test(Teacher, criterion = nn.CrossEntropyLoss())
test_loss /= (split*sub)

print('Result of testing teacher model')
print('false positive (FP): {}, false negative (FN): {}, true positive (TP): {}, true negative (TN): {}'.format(FP, FN, TP, TN))
print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%). Total time = {time() - start_time}')
print('Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%' .format(P, R, F1))


Result of testing teacher model
false positive (FP): 36, false negative (FN): 413, true positive (TP): 93, true negative (TN): 19457
Test set: Average loss: 0.0020, Accuracy: 97.75%). Total time = 2.636810064315796
Precision: 72.093%, Recall: 18.379%, F1-measure: 29.291%


In [7]:
noKD = train(noKD, train_loader, learning_rate = 0.0003, num_epochs = 200)
save_model(noKD, save_noKD_path)

Train Epoch: 1/200 [78400/79999 (100%)]  Loss: 1064.930625: 100%|██████████| 1600/1600 [00:05<00:00, 267.58it/s]
Train Epoch: 2/200 [78400/79999 (100%)]  Loss: 1065.762394: 100%|██████████| 1600/1600 [00:05<00:00, 289.58it/s]
Train Epoch: 3/200 [78400/79999 (100%)]  Loss: 951.789428: 100%|██████████| 1600/1600 [00:05<00:00, 290.60it/s]
Train Epoch: 4/200 [78400/79999 (100%)]  Loss: 526.150686: 100%|██████████| 1600/1600 [00:05<00:00, 290.74it/s]
Train Epoch: 5/200 [78400/79999 (100%)]  Loss: 383.664132: 100%|██████████| 1600/1600 [00:05<00:00, 281.29it/s]
Train Epoch: 6/200 [78400/79999 (100%)]  Loss: 346.021092: 100%|██████████| 1600/1600 [00:05<00:00, 289.86it/s]
Train Epoch: 7/200 [78400/79999 (100%)]  Loss: 324.975238: 100%|██████████| 1600/1600 [00:05<00:00, 287.59it/s]
Train Epoch: 8/200 [78400/79999 (100%)]  Loss: 311.821755: 100%|██████████| 1600/1600 [00:05<00:00, 291.23it/s]
Train Epoch: 9/200 [78400/79999 (100%)]  Loss: 301.268683: 100%|██████████| 1600/1600 [00:05<00:00, 29

In [8]:


start_time = time()
accuracy, test_loss, P, R, F1, TP, FP, TN, FN = test(noKD, criterion = nn.CrossEntropyLoss())
test_loss /= (split*sub)

print('Result of testing noKD model')
print('false positive (FP): {}, false negative (FN): {}, true positive (TP): {}, true negative (TN): {}'.format(FP, FN, TP, TN))
print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%). Total time = {time() - start_time}')
print('Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%' .format(P, R, F1))

Result of testing teacher model
false positive (FP): 1, false negative (FN): 504, true positive (TP): 2, true negative (TN): 19492
Test set: Average loss: 0.0022, Accuracy: 97.47%). Total time = 1.9545726776123047
Precision: 66.667%, Recall: 0.395%, F1-measure: 0.786%


In [10]:
from torch.nn import Parameter
from torch.nn.modules.module import Module
import torch.nn.functional as F
import math

save_student_path = ('../datasets/Spirit/model/chronological_student.pth')
def train_step(
    Teacher,
    Student,
    optimizer,
    student_loss_fn,
    divergence_loss_fn,
    temp,
    alpha,
    epoch,
    device
):
    losses = []
    pbar = tqdm(train_loader, total=len(train_loader), position=0, leave=True, desc=f"Epoch {epoch+1}")
    for data, targets in pbar:
        # Get data to cuda if possible
        data = data.to(device)
        targets = targets.to(device)

        # forward
        with torch.no_grad():
            teacher_preds, _ = Teacher(data)

        student_preds, __ = Student(data)
        student_loss = student_loss_fn(student_preds, targets)
        
        ditillation_loss = divergence_loss_fn(
            F.softmax(student_preds / temp, dim=1),
            F.softmax(teacher_preds / temp, dim=1)
        )
        loss = alpha * student_loss + (1 - alpha) * ditillation_loss
        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
    
    avg_loss = sum(losses) / len(losses)
    return avg_loss

def teach(epochs, Teacher, Student, temp=7, alpha=0.3):
  Teacher = Teacher.to(device)
  Student = Student.to(device)
  student_loss_fn = nn.CrossEntropyLoss()
  divergence_loss_fn = nn.KLDivLoss(reduction="batchmean")
  optimizer = torch.optim.Adam(Student.parameters(), lr=0.001)

  Teacher.eval()
  Student.train()
  for epoch in range(epochs):
      loss = train_step(
          Teacher,
          Student,
          optimizer,
          student_loss_fn,
          divergence_loss_fn,
          temp,
          alpha,
          epoch,
          device
      )

      print(f"Loss:{loss:.2f}")

Student = DistilLog(input_size = input_size, hidden_size=4, num_layers = 1, num_classes = num_classes, is_bidirectional=False).to(device)

Teacher = load_model(Teacher, save_teacher_path)
teach(epochs=100, Teacher=Teacher, Student=Student, temp=7, alpha=0.3)
save_model(Student, save_student_path)



Epoch 1: 100%|██████████| 1600/1600 [00:07<00:00, 227.20it/s]


Loss:-0.47


Epoch 2: 100%|██████████| 1600/1600 [00:07<00:00, 224.50it/s]


Loss:-0.57


Epoch 3: 100%|██████████| 1600/1600 [00:06<00:00, 229.21it/s]


...

Loss:-0.80


Epoch 99: 100%|██████████| 1600/1600 [00:07<00:00, 225.59it/s]


Loss:-0.80


Epoch 100: 100%|██████████| 1600/1600 [00:07<00:00, 224.48it/s]


Loss:-0.80


In [11]:
start_time = time()
accuracy, test_loss, P, R, F1, TP, FP, TN, FN = test(Student, criterion = nn.CrossEntropyLoss())
test_loss /= (split*sub)

print('Result of testing student model')
print('false positive (FP): {}, false negative (FN): {}, true positive (TP): {}, true negative (TN): {}'.format(FP, FN, TP, TN))
print(f'Test set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%). Total time = {time() - start_time}')
print('Precision: {:.3f}%, Recall: {:.3f}%, F1-measure: {:.3f}%' .format(P, R, F1))

Result of testing student model
false positive (FP): 108, false negative (FN): 441, true positive (TP): 65, true negative (TN): 19385
Test set: Average loss: 0.0101, Accuracy: 97.25%). Total time = 1.957085132598877
Precision: 37.572%, Recall: 12.846%, F1-measure: 19.146%
